## Task 5

1. Arranging the price data for these firms
2. Calculate the crash risk.

A stock market collapse is a sudden and unexpected decline in stock prices. A stock market fall can occur as a result of a large disastrous event, an economic crisis, or the bursting of a long-term speculative bubble.

Expanded market model regression 
$$r_{j,\tau} = \alpha_{j} + \gamma_{1,j}r_{m,\tau-2} + \gamma_{2,j}r_{m,\tau-1} + \gamma_{3,j}r_{m,\tau} + \gamma_{4,j}r_{m,\tau+1} + \gamma_{5,j}r_{m,\tau+2} + \epsilon_{j,\tau} \tag{1}$$

Where 
- $r_{j,\tau}$ be the return of $j$ th firm in $\tau$ th day
- $r_{m,\tau}$ be the return of Nifty50 in $\tau$ th day
- $\epsilon_{j,\tau}$ be the error term of $j$ th firm in $\tau$ th day

For the specific firm, the above regression equation would be
$$r_{\tau} = \alpha + \gamma_{1}r_{m,\tau-2} + \gamma_{2}r_{m,\tau-1} + \gamma_{3,j}r_{m,\tau} + \gamma_{4}r_{m,\tau+1} + \gamma_{5}r_{m,\tau+2} + \epsilon_{\tau} \tag{2}$$

To fit the above regression for a specific firm, required data format is given by 
firm return | index return before 2 day | index return before 1 day | index return at that day | index return after 1 day | index return after 2 day
---------|---------|---------|---------|---------|--------- 

The **firm specific daily** return for firm $j$ in day $\tau$ is calculated as the natural logarithm of one plus the residual return 
$$w_{j,\tau} = \ln(1+e_{j,\tau}) \tag{3}$$

Crash risk Measures

1. NSKEW (Negative skewness)

This measure captures the asymmetry of the return distribution of the firm. Negative (positive) values for the skewness indicate data that are skewed to the left (right). NSKEW is calculated by taking the negative of the third moment of firm specific daily returns for each uear and normalising it by the standard deviation firm-specific daily returns raised to the third power. This measure is multiplied by -1 so that a higher value corresponds to greater crash risk. For each firm $j$ in day $\tau$, 

$$NSKEW_{j, \tau} = -\frac{n(n-1)^{3/2} \sum_{\tau} w^3_{j,\tau}}{(n-1)(n-2)(\sum_{\tau} w^2_{j,\tau})^{3/2}}\tag{4}$$

2. DUVOL (Down-to-Top Volatility)

This is the down to top volatility measure of the crash likelihood. 
A higher value of the DUVOL indicated greater crash risk. DUVOL does not involve third moments and hence is less likely to be overly influenced by extreme daily returns.

$$DUVOL_{j,\tau} = \log \left(\frac{(n_u-1)\sum_{Down}w^2_{j,\tau}}{(n_d-1)\sum_{Up} w^2_{j,\tau}} \right) \tag{5}$$

![image.png](./../img/others/line_map.png)

Workflow for Cradit Risk Calculation:
- Make the appropriate table to run regression
- Regression has to be done and collect the residuals
- Calculate the quantity in equation (3)

1. Count
    - Count the firm-specific daily return falling below its (mean-3.09sd)
    - Count the firm-specific daily return falling above its (mean+3.09sd)
    - Difference between the above two
2. NSKEW
    - Use the information collected in first three steps
    - Making the function in equation (5)
3. DUVOL
    - Use the information collected in first three steps
    - Making the function in equation (6)

Construct static function in equation 4, 5 and 6

### Import Modules

In [29]:
import os, sys, copy, time
from tqdm.notebook import tqdm
import numpy as np
import pandas as pd
import pandas_datareader as get_data
import statsmodels.api as sm

In [19]:
# Load the data
micron_ltd = pd.read_csv('./../data/micron_ltd.csv')
print ('Shape: ', micron_ltd.shape)
micron_ltd.head()

Shape:  (558, 7)


,Unnamed: 0,index,co_code,company_name,co_stkdate,nse_closing_price,nse_returns
0,0,41393,11,20 MICRONS LTD.,2019-01-01,44.25,1.05
1,1,41394,11,20 MICRONS LTD.,2019-01-02,44.05,1.00
2,2,41395,11,20 MICRONS LTD.,2019-01-03,43.45,0.99
3,3,41396,11,20 MICRONS LTD.,2019-01-04,42.25,0.97
4,4,41397,11,20 MICRONS LTD.,2019-01-07,42.65,1.01


In [25]:
micron_ltd['co_stkdate'].iloc[0]

'2019-01-01'

In [28]:
market = get_data.DataReader('^NSEI', 'yahoo', start = micron_ltd['co_stkdate'].iloc[0], end = micron_ltd['co_stkdate'].iloc[-1])
market['Close'].pct_change()

Date
2019-01-02         NaN
2019-01-03   -0.011142
2019-01-04    0.005163
2019-01-07    0.004144
2019-01-08    0.002818
                ...   
2021-03-24   -0.017911
2021-03-25   -0.015430
2021-03-26    0.012733
2021-03-30    0.023285
2021-03-31   -0.010401
Name: Close, Length: 552, dtype: float64

### Define Functions

In [ ]:
# Function for making the data for regression
def making_data_reg(firm_CP, market_CP):
    '''
    Input: closing price of firm's stock, market stock
    Output: X and y
    '''
    df = pd.DataFrame()
    df['y'] = firm_CP.pct_change()

    market_return = market_CP.pct_change()
    idx = ['x'+i for i in ['-2','-1','0','+1','+2']]
    for i,j in zip(idx,range(-2,3,1)):
        df[i] = market_return.shift(j)
    return df

In [ ]:
# Fitting multiple regression and getting the error part
def regression(X, y):
    '''
    Input: Dependent variable and design matrix
    Output: Residual = (y-y_hat)
    '''
    beta_hat = np.linalg.inv(X.T @ X)@X.T@y
    y_hat = X@beta_hat
    return y-y_hat

In [ ]:
# Function to calculate firm-specific daily return
def firm_specific_return(residuals):
    '''
    Input: Residuals (e_tau)
    Output: w = ln(1+e_tau)
    '''
    return np.log(1 + residuals)

In [5]:
# First measure: Binary
def BINARY(w):
    '''
    Input: (x) firm-specific daily return
    Output: 0 or 1
    '''
    q_99 = np.mean(w)-3.09*np.std(w)
    if np.sum(w < q_99) > 0:
        CR = 1
    else:
        CR = 0
    return CR

# Second measure: NSKEW
def NSKEW(w):
    '''
    Input: (w) firm-specific daily return
    Output: -(n*(n-1)*np.sum(w**3))/((n-1)*(n-2)*(np.sum(w**2)**1.5))
    '''
    n = len(w)
    numerator = n*(n-1)*np.sum(w**3)
    denominator = (n-1)*(n-2)*(np.sum(w**2)**1.5)
    return -numerator/denominator

# Third measure: DUVOL
def DUVOL(w):
    '''
    Input: (w) firm-specific daily return
    Output: log(((n_u-1)*np.sum(w_downdays**2))/(n_d-1)*np.sum(w_updays**2))
    '''
    n = len(w)
    mean = np.mean(w)
    # select index of days having above or below the annual mean
    updays = np.argwhere(w > mean).flatten(); n_u = len(updays)
    downdays = np.argwhere(w < mean).flatten(); n_d = len(downdays)
    # getting returns of specific indices
    w_updays = w[updays]; w_downdays = w[downdays]
    numerator = (n_u-1)*np.sum(w_downdays**2)
    denominator = (n_d-1)*np.sum(w_updays**2)
    return np.log(numerator/denominator)

Problem faced

- There are no common null values in both the exchange
- How can I select one firm's data from one of the exchange?
- Some date issue is there. Like suppose I collect market data for duration first date and last date of the firm's available date, then length is getting small for the market data.
- Could we replace the NA values after taking the lagged market data. Otherwise multiple regression cannot be work.

Market data is collected from : https://www1.nseindia.com/products/content/equities/indices/historical_index_data.htm
Assumptions:
- Considered only those dates data where firms data is available.
- First we will try for those firms which have most data.